In [2]:
import sys
sys.path.append("../")


In [10]:
import warnings
warnings.filterwarnings('ignore')
import os
import pandas as pd
import numpy as np
from collections import defaultdict
from glob import glob
from common.evaluation import iter_thresholds
from sklearn.metrics import f1_score, precision_score, recall_score, roc_auc_score

In [51]:
def load_tuple(path, nrows=None):
    anomaly_score = np.load(
        os.path.join(path, "anomaly_score.npz"), allow_pickle=True
    )["arr_0"].item()["test"][:nrows]
    anomaly_label = np.load(os.path.join(path, "anomaly_label.npz"))[
        "arr_0"
    ].astype(int)[:nrows]
    minlen = min(len(anomaly_score), len(anomaly_label)) 
    return (anomaly_score[:minlen], anomaly_label[:minlen])

In [22]:
def compute_metrics(anomaly_score, anomaly_label):
    _, _, best_adjust_pred, _ = iter_thresholds(anomaly_score, anomaly_label, metric="f1", adjustment=True)
    _, _, best_raw_pred, _ = iter_thresholds(anomaly_score, anomaly_label, metric="f1", adjustment=False)
    
    aF1 = f1_score(anomaly_label, best_adjust_pred)
    aPC = precision_score(anomaly_label, best_adjust_pred)
    aRC = recall_score(anomaly_label, best_adjust_pred)
    
    rF1 = f1_score(anomaly_label, best_raw_pred)
    rPC = precision_score(anomaly_label, best_raw_pred)
    rRC = recall_score(anomaly_label, best_raw_pred)
    return [aF1, aPC, aRC, rF1, rPC, rRC] 

def concat_keys(value_dict, keys):
    anomaly_score_con = np.concatenate([value_dict[k]["anomaly_score"] for k in keys])
    anomaly_label_con = np.concatenate([value_dict[k]["anomaly_label"] for k in keys])
    return anomaly_score_con, anomaly_label_con

In [52]:
summarize_dict = {
    "Group A": {"machine-1-1": 38},
    "Group B": {"machine-1-2": 38},
    "Group C": {"machine-1-3": 38}
}
# summarize_dict = {"Group A": {"e29ca1cd": 3,
# "c23b2b2d": 4,
# "aeb5a1de": 6,
# "2fe95315": 5,
# "0a82a873": 7,
# "af732cc4": 7},
# "Group B": {"b2a04b7f": 20,
# "c2970798": 13,
# "5dafb960": 12},
# "Group C": {"c91f4a07": 24,
# "ca2ae31d": 43,
# "f7958fb7": 37}}
results_dir = "../benchmark/benchmarking_results/"

In [53]:
nrows = 100
target_dataset = "SMD"
final_res = []
# models = ["3sigma", "iforest",  "LODA", "PCA", "AutoEncoder", "lstm", "lstm_vae", "dagmm", "omnianomaly", "CMAnomaly_old",]
models = ["3sigma", "AutoEncoder"]
for model in models:
    df_dict = {}
    folders = glob(os.path.join(results_dir, f"{model}/*/*/*"))
    subdataset_values = {}
    for folder in folders:
        folder_components = folder.split(os.sep)
        dataset, subdataset = folder_components[-2], folder_components[-1]
        if dataset != target_dataset:
            continue
        anomaly_score, anomaly_label = load_tuple(folder, nrows)
        subdataset_values[subdataset] = {
            "anomaly_score": anomaly_score,
            "anomaly_label": anomaly_label,
        }

    for group_name, subdatasets in summarize_dict.items():
        df_dict = {}
        df_dict["Group"] = group_name
        df_dict["model"] = model
        df_dict["dataset"] = target_dataset
        
        subdatasets = list(subdatasets.keys())

        anomaly_score_con, anomaly_label_con = concat_keys(subdataset_values, subdatasets)
        aF1_con, aPC_con, aRC_con, rF1_con, rPC_con, rRC_con = compute_metrics(anomaly_score_con, anomaly_label_con)

        df_dict["aF1_con"] = aF1_con
        df_dict["aPC_con"] = aPC_con
        df_dict["aRC_con"] = aRC_con
        df_dict["rF1_con"] = rF1_con
        df_dict["rPC_con"] = rPC_con
        df_dict["rRC_con"] = rRC_con

        metrics_save = defaultdict(list)
        for subdataset in subdatasets:
            anomaly_score, anomaly_label = subdataset_values[subdataset]["anomaly_score"], subdataset_values[subdataset]["anomaly_label"]
            aF1, aPC, aRC, rF1, rPC, rRC = compute_metrics(anomaly_score, anomaly_label)
            metrics_save["aF1"].append(aF1)
            metrics_save["aPC"].append(aPC)
            metrics_save["aRC"].append(aRC)
            metrics_save["rF1"].append(rF1)
            metrics_save["rPC"].append(rPC)
            metrics_save["rRC"].append(rRC)
        df_dict.update({k: sum(v)/len(v) for k,v in metrics_save.items()}) 

        final_res.append(df_dict)
    print(f"Finish model {model}")
result_df = pd.DataFrame(final_res, columns = ["model","dataset","Group","aF1_con","aF1", "rF1_con", "rF1", "aPC_con","aRC_con","aPC","aRC","rPC_con","rRC_con","rPC","rRC"])
result_df.to_csv("final_results.csv", index=False)
result_df

Finish model 3sigma
Finish model AutoEncoder


,model,dataset,Group,aF1_con,aF1,rF1_con,rF1,aPC_con,aRC_con,aPC,aRC,rPC_con,rRC_con,rPC,rRC
0,3sigma,SMD,Group A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,3sigma,SMD,Group B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3sigma,SMD,Group C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,AutoEncoder,SMD,Group A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,AutoEncoder,SMD,Group B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,AutoEncoder,SMD,Group C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
